# Systemy uczące się - Zad. dom. 4: Boosting gradientowy

### Autor rozwiązania: Maciej Wieczorek, 148141

Ten notebook zawiera zadania związane z boostingiem gradientowym (Gradient Boosting)
Do notebooka zostały dołączony plik helpers.py , które są używane w zadaniach, 
nie musisz do nich zaglądać ani ich modyfikować. Notebook jest sprawdzany półautomatycznie - przed wysyłką sprawdź czy cały kod wykonuje się bez błędów.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Zadanie 1.

W tym zadaniu będziesz implementował(a) algorytm Gradient Boosting Trees dla problemu regresji. Aby zaimplementować ten algorytm dostępny jest obiekt `Node` implementujący drzewo regresyjne. Jest to odpowiednio dostosowany obiekt, który implementowałeś w pierwszym zadaniu domowym. Możesz wykorzystać swoją własną implementację (i dostosować ją wg. opisu poniżej) lub skorzystać z implementacji w pliku `helpers`.

W stosunku do poprzedniej implementacji obiekt ma pewne dodatkowe cechy, które umożliwią sprawniejszą implementację:
- W konstruktorze `Node` jest teraz jeden obowiązkowy argument `calculate_leaf_value`, do którego należy wstawić funkcję, która jest wywoływana przez algorytm w momencie tworzenia liścia celem obliczenia jego wartości. W standardowym drzewie regresji, algorytm tworzący liść oblicza jego wartość jako średnią wartość jego elementów. Jeśli chcielibyśmy uzyskać takie działanie powinniśmy zaimplementować następującą funkcję:
```python
def mean_val_leaf(X, y, last_predicted):
    return np.mean(y)

tree = Node(calculate_leaf_value=mean_val_leaf)
```
Zwróć uwagę na parametry funkcji tworzącej lisć: `X`, `y` charakteryzujące obiekty w liściu oraz `last_predicted` przechowujące aktualną predykcję klasyfikatora dla tych obiektów. Poprzez aktualną predykcję rozumiemy tu predykcję uzyskaną wszystkimi dotychczas stworzonymi klasyfikatorami bazowymi w GBT (czyli wynik osiągnięty pozostałymi drzewami niż to tworzone). Argument `last_predicted` na chwilę obecną wydaje się niepotrzebny lecz będzie on użyty do realizacji zadania.

- Dodatkowe argumenty obsługuje też funkcja ucząca model `fit(X, y, last_predicted, max_depth = None)` - która dostaje na wejście wcześniej wspomiane `last_predicted` oraz argument `max_depth` wstrzymujący budowę zbyt głębokich drzew. Innych mechanizmów pruningu niezaimplementowano, jeśli jednak takowe istnieją w Twojej implementacji, możesz je wykorzystać.

Stwórz zbiór danych do regresji poniższym kodem:

In [ ]:
X = np.random.uniform(-5,5,100)
y = 5 + X + np.sin(X) + np.random.normal(scale=0.1, size=100)
plt.plot(X,y,'o')
X = np.expand_dims(X, axis=1)

Zaimplementuj algorytm GBT dla błędu kwadratowego. Aby to zrobić należy uzupełnić w ogólnym pseudokodzie przedstawionym na zajęciach trzy elementy:
- model początkowy $F_0(x)$ zwracający stałą wartość $v$ która optymalizuje błąd:
$$F_0(x) = \arg \min_v \sum_{i=1}^N L(y_i, v) $$
- wzór na wartość ujemnego gradientu tj. pseudo-rezyduum:
$$r_i  =  - \frac{\partial}{\partial \hat{y_i}} L(y_i, \hat{y_i}) $$
gdzie $\hat{y_i}$ to aktualna predykcja klasyfikatora tj. w $m$-tej iteracji $\hat{y_i}=F_m(x)$
- wzór na wartość liścia $v$ optymalizujący funkcję celu całego modelu GBT
$$v = \arg \min_v \sum_{i=1}^{N_l} L(y_i, F_{m-1}(x_i) + v) $$
Zwróć uwagę, że suma iteruje tylko po instancjach w liściu (${N_l}$ to liczba elementów w liściu).

Wyznacz powyższe wartości (rozwiązania dla referencji znajdziesz poniżej komórek z kodem) i zaimplementuj algorytm. 

In [ ]:
from helpers import Node

class GBTRegressor(object):
    def __init__(self, lr=1.0):
        self.lr = lr
        self.initial_model = None
        self.trees = []
        self.residuals = []
        
    def fit(self, X, y, M = 1000, max_depth = 1):
        """
		Argumenty:
		X -- numpy.ndarray, macierz cech o wymiarach (n, m), gdzie n to liczba przykładów, a m to liczba cech
		y -- numpy.ndarray, wektor klas o długości n, gdzie n to liczba przykładów
        M -- int, liczba drzew
        max_depth -- int, maksymalna głębokość pojedynczego drzewa
		"""
        # TWÓJ KOD TUTAJ
            
    def predict(self, X):
        """
		Argumenty:
		X -- numpy.ndarray, macierz cech o wymiarach (n, m), gdzie n to liczba przykładów, a m to liczba cech, dla których chcemy dokonać predykcji

		Wartość zwracana:
		numpy.ndarray, wektor przewidzoanych klas o długości n, gdzie n to liczba przykładów
        """
        Y = np.array([f.predict(X) for f in self.trees])
        return self.initial_model + self.lr * Y.sum(0)

Przetestuj działanie algorytmu:

In [ ]:
X_test = np.linspace(-5, 5, num=500)
X_test = np.expand_dims(X_test, axis=1)

gbt = GBTRegressor()
gbt.fit(X, y)
y_pred = gbt.predict(X_test)

plt.plot(X,y,'o')
plt.plot(X_test,y_pred,'-')

Narysuj wynik modelu z odpowednio 1, 2, 5, 10 i 100 klasyfikatorami bazowami. Za klasfikator bazowy przyjmij decision stump.

In [ ]:
plt.plot(X, y, 'o', label='test')

for m in [1, 2, 5, 10, 100]:
    # TWÓJ KOD TUTAJ
    
    plt.plot(X_test, Y_test, '-', label=f'M = {m}')

plt.legend(loc='upper left')

Sprawdź jak zmieniają się wartości rezyduów w kilku początkowych iteracjach GBT. Narysuj wykresy $x$ vs $y-\hat{y}$ - zwróć uwagę, że tak właśnie wyglądają zbiory na których uczą się kolejne klasyfikatory.

In [ ]:
gbt = GBTRegressor()
M = 10
gbt.fit(X, y, M=M)
cm = plt.get_cmap('viridis')
plt.vlines(X, ymin=y, ymax=y+gbt.residuals[-1], colors='r', ls='dotted', label='last residual')
for i in range(M):
    color = cm(np.ones(len(X)) * i/(M-1))
    plt.scatter(X, y + gbt.residuals[i], c=color, s=10, label=f'iter = {i+1}')
plt.scatter(X, y, c='r', s=50, marker='*', label='ground truth')
plt.legend()

*Odpowiedzi:*
- model początkowy $F_0(x)$
$$F_0(x) = \arg \min_v \sum_{i=1}^N L(y_i, v) = \frac{1}{2} \sum_{i=1}^N (y_i- v)^2 $$
wartość ta to oczywiście średnia arytmetyczna $v = \frac{1}{n} \sum_{i=1}^N y_i$. (Upewnij się, że to rozumiesz poprzez policzenie pochodnej i przyrównanie jej do 0).
- wzór na wartość ujemnego gradientu tj. pseudo-rezyduum 
$$r_i  =  - \frac{\partial}{\partial \hat{y_i}} L(y_i, \hat{y_i}) = - \frac{\partial}{\partial \hat{y_i}} \frac{1}{2}(y_i- \hat{y_i})^2$$
Co po przekształceniach wykorzystujących regułę łańcuchową ("pochodna zewnętrzna razy pochodna wewnętrzna"):
$$r_i  = -\frac{1}{2} 2(y_i- \hat{y_i})\frac{\partial}{\partial \hat{y_i}} (y_i- \hat{y_i}) 
= -(y_i- \hat{y_i})\cdot(-1)
= y_i- \hat{y_i}  $$
- wzór na wartość liścia $v$ optymalizujący funkcję celu
$$v = \arg \min_v \sum_{i=1}^{N_l} L(y_i, F_{m-1}(x_i) + v) = \frac{1}{2} \sum_{i=1}^{N_l} (y_i- F_{m-1}(x_i) - v)^2 $$
Co można dalej obliczyć poprzez przyrównanie pochodnej do 0 lub poprzez zauważenie że jest to w naszej sytuacji ten sam wzór co dla modelu początkowego gdzie $y_i$ zostało zastępione $y_i- F_{m-1}(x_i)=r_i$. W związku z tym wartość liścia to $v = \frac{1}{n} \sum_{i=1}^N r_i$

## Zadanie 2.

Zaimplementuj GBT dla problemu klasyfikacji binarnej, który będzie optymalizował błąd regresji logicznej tj. entropię krzyżową wyrażoną wzorem:
$$L(y_i, \hat{p_i}) = y_i \log \hat{p_i} +  (1-y_i) \log (1-\hat{p_i}) $$
gdzie $y_i\in \{0,1\}$ to prawdziwa wartość klasy a $\hat{p_i}$ to predykcja klasyfikatora dla $i$-tego elementu.

- Zauważ, że GBT wykorzystuje drzewa regresji, które - choć modyfikujemy im sposób obliczania liści - nadal tworzą podziały dla miary SSE. Aby wykorzystać GBT do problemu klasyfikacji należy zastanowić się jak możemy wykorzystać regresor do klasyfikacji. Ten problem rozwiązywaliśmy już wcześniej przy omawianiu regresji logistycznej, gdzie tworzyliśmy klasyfikator z modelu regresji liniowej. Przypomnijmy, że w regresji logistycznej model regresji liniowej służy do predykcji logitu prawdopodobieństwa klasy (który przypomnijmy ma zakres wartości od $-\infty$ do $\infty$)
$$\text{logit}(p_x) = \ln \frac{p_x}{1-p_x}=w^Tx+b$$
Podobnie w GBT należy skonstruować model regresji do przewidywania wartości $\text{logit}(p_x)$, a jedynie przy predykcji (lub kiedy jest to wygodne) transformować go do prawdopodobieństwa klasy funkcją sigmoidalną $p_x  = \frac{1}{1+e^{- \text{logit}(p_x)}}  $

**Zadania**

1. Powyższy zapis funkcji celu $L(y_i, \hat{p_i})$ jest wyrażony w zależności od prawdopodobieństwa klasy, a nie wartości logitu $L(y_i, \text{logit}(\hat{p_i}))$. Przekształć wzór na funkcję celu, aby jej argumentem był logit. Zwróć uwagę, że model regresji będzie przewidywał właśnie logit, więc przy wyznaczaniu elementów algorytmu GBT należy liczyć np. pochodne tej właśnie przekształconej funkcji.

    Zapisz wzór na tę funkcję w komórce poniżej:


$$\hat y_i \equiv \text{logit}(\hat p_i)$$

$$\hat p_i = \frac{ 1 }{ 1 + e^{- \hat y_i} } = \frac{ e^{\hat y_i} }{ 1 + e^{\hat y_i} }$$

$$L(y_i, \hat y_i) = ?$$

2. Zacznijmy uzupełniać w ogólnym pseudokodzie przedstawionym na zajęciach brakujące elementy. Wyznacz model początkowy $F_0(x)$ zwracający stałą wartość $v$ która optymalizuje błąd:
$$F_0(x) = \arg \min_v \sum_{i=1}^N L(y_i, v) $$

$$\sum_{i=1}^n \frac{\partial}{\partial v} L(y_i, v) = 0$$

3. Wyznacz wzór na wartość ujemnego gradientu tj. pseudo-rezyduum:

$$r_i  =  - \frac{\partial}{\partial \hat{y_i}} L(y_i, \hat{y_i}) $$
Uwaga 1: na samym końcu, aby wzór uzykał prostszą formę, możesz zamienić w nim wartości logitów z powrotem na prawdopodobieństwa. 

Uwaga 2: musiałeś policzyć go ju w poprzednim punkcie.

$$r_i = ?$$

4. Wzór na wartość liścia $v$ optymalizujący funkcję celu całego modelu GBT
$$v = \arg \min_v \sum_{i=1}^{N_l} L(y_i, F_{m-1}(x_i) + v) $$
niestety nie jest prosty do wyznaczenia w tym przypadku. Stosuje się przybliżenie Taylora drugiego rzedu tej funkcji i wtedy optimum ma postać:
$$v = \frac{-\sum_{i=1}^{N_L} L_i' }{\sum_{i=1}^{N_L} L_i''}$$
gdzie $L_i'$ i $L_i''$ to skrócony zapis pierwszej i drugiej pochodnej policzonej po funkcji straty dla $i$-tego elementu. Ponieważ $r_i=-L_i'$ to licznik przyjmuje postać $\sum_{i=1}^{N_L} r_i $. Wyznacz cały wzór.



$$\hat{y_i} \equiv F_{m-1}(x_i)$$

$$
v = \frac{-\sum_{i=1}^n \frac{\partial}{\partial v} L(y_i, \hat{y_i} + v)}{\sum_{i=1}^n \frac{\partial^2}{\partial^2 v} L(y_i, \hat{y_i} + v)} = ?
$$

Wykorzystując uzyskane wyniki zaimplementuj algorytm. 

In [ ]:
import scipy.special
# Wskazówka: scipy.special.expit() implemenuje funkcję sigmoidalną

class GBTClassifier(object):
    def __init__(self, lr=1.0):
        self.lr = lr
        self.initial_model = None
        self.trees = []
        self.residuals = []
        
    def fit(self, X, y, M=100, max_depth=1):
        """
		Argumenty:
		X -- numpy.ndarray, macierz cech o wymiarach (n, m), gdzie n to liczba przykładów, a m to liczba cech
		y -- numpy.ndarray, wektor klas o długości n, gdzie n to liczba przykładów
        M -- int, liczba drzew
        max_depth -- int, maksymalna głębokość pojedynczego drzewa
		"""
        # TWÓJ KOD TUTAJ
            
    def predict(self, X):
        """
		Argumenty:
		X -- numpy.ndarray, macierz cech o wymiarach (n, m), gdzie n to liczba przykładów, a m to liczba cech, dla których chcemy dokonać predykcji

		Wartość zwracana:
		numpy.ndarray, wektor przewidzoanych klas o długości n, gdzie n to liczba przykładów
        """
        Y = np.array([f.predict(X) for f in self.trees])
        return scipy.special.expit(self.initial_model + self.lr * Y.sum(0))

Przetestuj swoją implementację na zbinaryzowanym zbiorze `iris`.

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data[:, [0, 2]]
y = iris.target
y[y==2] = 0 # Sprowadzenie problemu do klasyfikacji binarnej

def draw_boundary(X, y, M=100, max_depth=1, lr=1, ax=None):
    # Kod rysowania
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                         np.arange(y_min, y_max, 0.1))

    clf = GBTClassifier(lr=lr)
    clf.fit(X, y, M=M, max_depth=max_depth)
    Y = clf.predict(X)
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    ax.contourf(xx, yy, Z, alpha=0.4)
    ax.scatter(X[:, 0], X[:, 1], c=y, s=20, edgecolor='k')
    loss = -np.mean(y*np.log(Y) + (1 - y)*np.log(1 - Y))
    ax.set_title(f'M={M}, max_depth={max_depth}, lr={lr}, L={loss:.4f}')
    
fig, ax = plt.subplots()
draw_boundary(X, y, M=100, max_depth=1, ax=ax)

Narysuj granice decyzji klasyfikatora dla 10, 20, 50 i 100 iteracji algorytmu dla klasyfikatora bazowego o maksymalnej głębokości 1 i 2.

In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=3, figsize=(15, 20))
for i, M in enumerate([10, 20, 50, 100]):
    for j, max_depth in enumerate([1, 2]):
        draw_boundary(X, y, M=M, max_depth=max_depth, ax=axs[i][j])

**Polecenia**

1. W jaki sposób zaimplementować GBT dla problemu klasyfikacji wieloklasowej?
2. W powyższym problemie który z klasyfikatorów bazowych (o jakiej max. głębokości) poradził sobie lepiej? Czy jest sens stosować w tym problemie drzewa o głębokości większej niż testowana (tj. 2). Odpowiedź uzasadnij.
3. Dodaj do implementacji parametr $\eta$ i przetestuj kilka jego wartości. Pamętaj, że $\eta$ powinno być wykorzystywane nie tylko w funkcji `fit`, ale także `predict` - dlaczego?

Odpowiedź na drugą kropkę umieść poniżej.

(MIEJSCE NA TWOJE ODPOWIEDZI)

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2)
for ax, lr in zip(axs.reshape(-1), [1, 0.5, 0.1, 0.01]):
    draw_boundary(X, y, M=100, max_depth=1, lr=lr, ax=ax)

## Zadanie 3.

GBT jest popularnym algorytmem między innymi dzięki bardzo efektywnym implementacjom potrafiącym sobie radzić z dużymi zbiorami danych. W tym ćwiczeniu Twoim zadaniem jest nauczenie się podstaw obsługi biblioteki `catboost`, którą powinieneś zainstalować.

In [ ]:
!pip install catboost

Wczytanie zbioru danych `iris` z poprzedniego zadania.

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data[:, [0, 2]]
y = iris.target
y[y==2] = 0 # Sprowadzenie problemu do klasyfikacji binarnej

Trening modelu

In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(logging_level='Silent')
model.fit(X, y, eval_set=(X, y), plot=True)

Przykładowy kod ewaluuje działanie algorytmu na części uczącej. Podziel zbiór na część uczącą i testową i ponownie uruchom algorytm. 

In [ ]:
from sklearn.model_selection import train_test_split

X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2)
model = CatBoostClassifier(logging_level='Silent')
model.fit(X_tr, y_tr, eval_set=(X_te, y_te), plot=True)

*Dla chętnych*: porównaj wartość funkcji straty osiągniętej przez catboost z Twoją implementacją z zadania 2.

Zaimportuj dowolny większy i bardziej wymagający zbiór danych. Ćwiczenie możesz wykonać na [dowolnym zbiorze danych](https://catboost.ai/docs/concepts/python-reference_datasets.html) - ładowanie zbioru może trochę potrwać. Jeśli masz problemy sprzętowe z operowaniem na dużym zbiorze danych to jest też dostępny zbiór `titanic`.

In [ ]:
from catboost import datasets

ds_tr, ds_te = datasets.titanic()

# TWÓJ KOD TUTAJ

Spróbuj osiągnąć jak najlepszy wynik na wybranym zbiorze poprzez tuning parametrów. Ważne parametry uczenia zostały opisane [tutaj](https://catboost.ai/docs/concepts/python-reference_parameters-list.html).

In [ ]:
model = CatBoostClassifier(logging_level='Silent')

grid = dict(
    learning_rate=[0.1, 1.0],
    depth=[4, 6, 10],
    # ???
)
result = model.randomized_search(grid, X_tr, y_tr, plot=True)
# `model` może być teraz użyty do predykcji 
print(result.params)